"""
Pipeline de predicción de retrasos (flights_clean.csv)
- Incluye: carga segura, features básicas, target encoding (TE), agregados históricos sin fuga,
split temporal, entrenamiento (LightGBM, XGBoost y RandomForest opcional memoria-seguro),
selección de umbrales, guardado de artefactos y función de inferencia/score.
- Cada bloque está numerado y comentado.


Requisitos mínimos del entorno (ya usados durante las pruebas previas):
pip install pandas numpy scikit-learn lightgbm xgboost joblib
(omitir CatBoost por problemas de build en Windows/py3.13)
"""

In [5]:
# # ============================================================
# # Predicción de retrasos de vuelos (2015) - Pipeline completo
# # ============================================================
# # Qué hace este script:
# # 1) Carga flights_clean.csv con dtypes compactos (memoria)
# # 2) Respeta features ya pre-calculadas (MINUTO_DIA_SALIDA, SALIDA_SIN/COS, etc.)
# # 3) Agrega MONTH_SIN/COS si faltan
# # 4) Construye DISTANCIA_HAV si falta (haversine)
# # 5) Arma RUTA (ORIGIN_AIRPORT->DESTINATION_AIRPORT) si falta
# # 6) Split temporal: train (MONTH 1-9), valid (MONTH 10-12)
# # 7) Target Encoding (TE) sin fuga + Agregados históricos (AIR/DES/ORI/RUTA/RUTA_HORA)
# # 8) Entrena LightGBM y XGBoost (RF opcional) con pesos por desbalance
# # 9) Calcula umbral F1 óptimo + umbrales operativos (precision~0.30, recall>=0.70)
# # 10) Guarda artefactos (modelos, TE, agregados, feature_order, metadata)
# # 11) Función de inferencia score_retraso(df_raw) lista para usar
# # ============================================================

# import os, json, time, math, gc, warnings
# warnings.filterwarnings("ignore")

# import numpy as np
# import pandas as pd
# from joblib import dump, load

# # ===========
# # Utilidades
# # ===========
# def downcast_int(series: pd.Series, signed=True):
#     """Reduce dtype de enteros para ahorrar RAM (sin alterar valores)."""
#     if not pd.api.types.is_integer_dtype(series):
#         return series
#     return pd.to_numeric(series, downcast="integer" if signed else "unsigned")

# def downcast_float(series: pd.Series):
#     """Reduce dtype de floats para ahorrar RAM."""
#     if not pd.api.types.is_float_dtype(series):
#         return series
#     return pd.to_numeric(series, downcast="float")

# def haversine(lat1, lon1, lat2, lon2):
#     """Distancia Haversine (km)."""
#     R = 6371.0
#     lat1 = np.radians(lat1.astype("float64"))
#     lon1 = np.radians(lon1.astype("float64"))
#     lat2 = np.radians(lat2.astype("float64"))
#     lon2 = np.radians(lon2.astype("float64"))
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
#     c = 2 * np.arcsin(np.sqrt(a))
#     return (R * c).astype("float32")

# # ===================================
# # 1) Carga segura + dtypes compactos
# # ===================================
# def load_flights(path_csv: str) -> pd.DataFrame:
#     """
#     Carga flights_clean.csv sin alterar columnas ya calculadas.
#     Hace downcast de tipos numéricos cuando es seguro.
#     """
#     t0 = time.time()
#     df = pd.read_csv(path_csv)
#     # Downcast num
#     for c in df.select_dtypes(include=["int64","int32","int16","int8"]).columns:
#         df[c] = downcast_int(df[c])
#     for c in df.select_dtypes(include=["float64","float32"]).columns:
#         df[c] = downcast_float(df[c])
#     # Categóricas: como string (evitar category por ahora para TE)
#     for c in ["AIRLINE","ORIGIN_AIRPORT","DESTINATION_AIRPORT","AIRLINE_NAME",
#               "ORIGEN_AEROPUERTO","ORIGEN_CIUDAD","ORIGEN_ESTADO",
#               "DEST_AEROPUERTO","DEST_CIUDAD","DEST_ESTADO","MOTIVO_RETRASO"]:
#         if c in df.columns:
#             df[c] = df[c].astype("string")

#     print(f"✓ Cargado: {df.shape} | en {time.time()-t0:.1f}s")
#     if "RETRASADO_LLEGADA" in df.columns:
#         print("✓ Rate retraso:", df["RETRASADO_LLEGADA"].mean())
#     return df

# # ============================
# # 2) RUTA (si no existe)
# # ============================
# def add_route(df: pd.DataFrame) -> pd.DataFrame:
#     out = df.copy()
#     if "RUTA" not in out.columns:
#         if {"ORIGIN_AIRPORT","DESTINATION_AIRPORT"}.issubset(out.columns):
#             out["RUTA"] = (out["ORIGIN_AIRPORT"].astype("string") + "->" +
#                            out["DESTINATION_AIRPORT"].astype("string"))
#         else:
#             raise KeyError("Faltan ORIGIN_AIRPORT/DESTINATION_AIRPORT para construir RUTA.")
#     return out

# # =====================================================
# # 3) Features de tiempo (respetando las columnas ya existentes)
# # =====================================================
# def ensure_time_features(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Respeta: MINUTO_DIA_SALIDA, HORA_SALIDA, SALIDA_SIN, SALIDA_COS.
#     Solo crea lo que falte. Agrega MONTH_SIN/COS si no existen.
#     """
#     out = df.copy()

#     # MINUTO_DIA_SALIDA
#     if "MINUTO_DIA_SALIDA" not in out.columns:
#         def _hhmm_to_minute(val):
#             if pd.isna(val): return np.nan
#             try:
#                 v = int(str(val).strip().split('.')[0])
#             except Exception:
#                 return np.nan
#             if v == 2400:  # normalizar 24:00 a 00:00
#                 return 0
#             hh, mm = v // 100, v % 100
#             hh = min(max(hh, 0), 23); mm = min(max(mm, 0), 59)
#             return hh * 60 + mm

#         if "SCHEDULED_DEPARTURE" in out.columns:
#             out["MINUTO_DIA_SALIDA"] = out["SCHEDULED_DEPARTURE"].map(_hhmm_to_minute).astype("float32")
#         elif "HORA_SALIDA" in out.columns:
#             out["MINUTO_DIA_SALIDA"] = (out["HORA_SALIDA"].astype("float32").clip(0, 23) * 60).astype("float32")
#         else:
#             raise KeyError("Falta 'MINUTO_DIA_SALIDA' y no puedo derivarlo (no hay SCHEDULED_DEPARTURE ni HORA_SALIDA).")

#     # HORA_SALIDA
#     if "HORA_SALIDA" not in out.columns:
#         out["HORA_SALIDA"] = (out["MINUTO_DIA_SALIDA"] // 60).astype("int16").clip(0, 23)

#     # SALIDA_SIN/COS
#     if "SALIDA_SIN" not in out.columns or "SALIDA_COS" not in out.columns:
#         mins = out["MINUTO_DIA_SALIDA"].astype("float32")
#         rad = 2.0 * np.pi * (mins / (24.0 * 60.0))
#         if "SALIDA_SIN" not in out.columns:
#             out["SALIDA_SIN"] = np.sin(rad).astype("float32")
#         if "SALIDA_COS" not in out.columns:
#             out["SALIDA_COS"] = np.cos(rad).astype("float32")

#     # MONTH_SIN/COS
#     if "MONTH" in out.columns:
#         if "MONTH_SIN" not in out.columns or "MONTH_COS" not in out.columns:
#             m = out["MONTH"].astype("float32")
#             ang = 2.0 * np.pi * (m - 1.0) / 12.0
#             if "MONTH_SIN" not in out.columns:
#                 out["MONTH_SIN"] = np.sin(ang).astype("float32")
#             if "MONTH_COS" not in out.columns:
#                 out["MONTH_COS"] = np.cos(ang).astype("float32")

#     # dtypes compactos
#     out["MINUTO_DIA_SALIDA"] = out["MINUTO_DIA_SALIDA"].astype("float32", copy=False)
#     out["HORA_SALIDA"]       = out["HORA_SALIDA"].astype("int16", copy=False)
#     out["SALIDA_SIN"]        = out["SALIDA_SIN"].astype("float32", copy=False)
#     out["SALIDA_COS"]        = out["SALIDA_COS"].astype("float32", copy=False)
#     if "MONTH_SIN" in out.columns: out["MONTH_SIN"] = out["MONTH_SIN"].astype("float32", copy=False)
#     if "MONTH_COS" in out.columns: out["MONTH_COS"] = out["MONTH_COS"].astype("float32", copy=False)
#     return out

# # =====================================================
# # 4) Distancia Haversine (si falta DISTANCIA_HAV)
# # =====================================================
# def ensure_distance(df: pd.DataFrame) -> pd.DataFrame:
#     out = df.copy()
#     if "DISTANCIA_HAV" not in out.columns:
#         need = {"ORIGEN_LAT","ORIGEN_LON","DEST_LAT","DEST_LON"}
#         if need.issubset(out.columns):
#             out["DISTANCIA_HAV"] = haversine(out["ORIGEN_LAT"], out["ORIGEN_LON"],
#                                              out["DEST_LAT"],  out["DEST_LON"])
#         elif "DISTANCE" in out.columns:
#             # fallback: usar DISTANCE (millas -> km aprox.)
#             out["DISTANCIA_HAV"] = (out["DISTANCE"].astype("float32") * 1.60934).astype("float32")
#         else:
#             raise KeyError("No puedo construir DISTANCIA_HAV (faltan lat/lon y DISTANCE).")
#     out["DISTANCIA_HAV"] = out["DISTANCIA_HAV"].astype("float32", copy=False)
#     return out

# # =====================================================
# # 5) Target Encoding (TE) sin fuga (train only)
# # =====================================================
# def build_target_encoding(df_train: pd.DataFrame, cols, target="RETRASADO_LLEGADA", smooth=20):
#     """
#     Devuelve:
#       - mappings: dict col -> {categoria: media suavizada}
#       - defaults: dict col -> media global (para unseen)
#       - global_mean: float
#     """
#     mappings, defaults = {}, {}
#     global_mean = df_train[target].mean()
#     for c in cols:
#         g = df_train.groupby(c, observed=False)[target]
#         m = g.mean()
#         n = g.size()
#         te = (m*n + global_mean*smooth) / (n + smooth)
#         mappings[c] = te.to_dict()
#         defaults[c] = float(global_mean)
#     return mappings, defaults, float(global_mean)

# def apply_target_encoding(df: pd.DataFrame, mappings, defaults, suffix="_TE"):
#     out = df.copy()
#     for c, mp in mappings.items():
#         newc = f"{c}{suffix}"
#         out[newc] = out[c].map(mp).astype("float32")
#         # unseen -> default
#         out[newc] = out[newc].fillna(defaults[c]).astype("float32")
#     return out

# # =====================================================
# # 6) Agregados históricos sin fuga (train only)
# # =====================================================
# def build_agg(df_train: pd.DataFrame, keys, target="RETRASADO_LLEGADA", pref="AIR", smooth=20, global_mean=None):
#     """
#     Por 'keys' (lista de columnas), calcula rate suavizado y conteo.
#     """
#     if global_mean is None:
#         global_mean = df_train[target].mean()
#     g = df_train.groupby(keys, observed=False)[target]
#     m = g.mean()
#     n = g.size()
#     rate = (m*n + global_mean*smooth) / (n + smooth)
#     agg = rate.rename(f"{pref}_rate").reset_index()
#     agg = agg.merge(n.rename(f"{pref}_n").reset_index(), on=keys, how="left")
#     return agg

# def left_join_agg(X: pd.DataFrame, agg: pd.DataFrame, keys, pref, global_mean):
#     out = X.merge(agg, on=keys, how="left")
#     # fill: rate -> global_mean; n -> 0
#     out[f"{pref}_rate"] = out[f"{pref}_rate"].fillna(global_mean).astype("float32")
#     out[f"{pref}_n"]    = out[f"{pref}_n"].fillna(0).astype("float32")
#     return out

# # =====================================================
# # 7) Construcción de matrices (base -> agregados -> TE)
# # =====================================================
# BASE_FEATURES = [
#     "MONTH","DAY_OF_WEEK","SALIDA_SIN","SALIDA_COS","MONTH_SIN","MONTH_COS",
#     "DISTANCIA_HAV","MINUTO_DIA_SALIDA","HORA_SALIDA"
# ]
# TE_COLS = ["AIRLINE","ORIGIN_AIRPORT","DESTINATION_AIRPORT","RUTA"]
# AGG_SPECS = [
#     (["AIRLINE"], "AIR"),
#     (["DESTINATION_AIRPORT"], "DES"),
#     (["ORIGIN_AIRPORT"], "ORI"),
#     (["RUTA"], "RUTA"),
#     (["RUTA","HORA_SALIDA"], "RUTA_HORA"),
# ]

# def make_matrices(df: pd.DataFrame):
#     """
#     Devuelve:
#       Xt, Xv, y_train, y_valid, mappings, defaults, global_mean, agg_tables (dict pref->df)
#     """
#     # target
#     if "RETRASADO_LLEGADA" not in df.columns:
#         raise KeyError("Falta la columna objetivo RETRASADO_LLEGADA.")
#     y = df["RETRASADO_LLEGADA"].astype("int8")

#     # split temporal (sin fuga)
#     train_mask = df["MONTH"].between(1, 9)
#     valid_mask = df["MONTH"].between(10, 12)

#     v_train = df.loc[train_mask].copy()
#     v_valid = df.loc[valid_mask].copy()
#     y_train = y.loc[train_mask].reset_index(drop=True)
#     y_valid = y.loc[valid_mask].reset_index(drop=True)

#     # X base
#     cols_needed = set(BASE_FEATURES + TE_COLS)
#     miss = cols_needed - set(df.columns)
#     if miss:
#         raise KeyError(f"Faltan columnas base/TE para matrices: {sorted(miss)}")

#     Xt = v_train[BASE_FEATURES + TE_COLS].reset_index(drop=True).copy()
#     Xv = v_valid[BASE_FEATURES + TE_COLS].reset_index(drop=True).copy()

#     # Agregados (train-only) -> join a Xt/Xv
#     global_mean = float(y_train.mean())
#     agg_tables = {}
#     for keys, pref in AGG_SPECS:
#         agg = build_agg(v_train, keys, target="RETRASADO_LLEGADA", pref=pref, smooth=20, global_mean=global_mean)
#         agg_tables[pref] = agg
#         Xt = left_join_agg(Xt, agg, keys, pref, global_mean)
#         Xv = left_join_agg(Xv, agg, keys, pref, global_mean)

#     # TE (build con v_train -> apply a Xt/Xv)
#     mappings, defaults, _gm = build_target_encoding(v_train, TE_COLS, target="RETRASADO_LLEGADA", smooth=20)
#     Xt = apply_target_encoding(Xt, mappings, defaults, suffix="_TE")
#     Xv = apply_target_encoding(Xv, mappings, defaults, suffix="_TE")

#     # Selección final de columnas (orden fijo = 23 features)
#     final_cols = [
#         "MONTH","DAY_OF_WEEK","SALIDA_SIN","SALIDA_COS","MONTH_SIN","MONTH_COS",
#         "DISTANCIA_HAV","MINUTO_DIA_SALIDA","HORA_SALIDA",
#         "AIRLINE_TE","ORIGIN_AIRPORT_TE","DESTINATION_AIRPORT_TE","RUTA_TE",
#         "AIR_rate","AIR_n","DES_rate","DES_n","ORI_rate","ORI_n",
#         "RUTA_rate","RUTA_n","RUTA_HORA_rate","RUTA_HORA_n"
#     ]
#     Xt = Xt[final_cols].astype("float32")
#     Xv = Xv[final_cols].astype("float32")

#     # liberar
#     del v_train, v_valid
#     gc.collect()

#     return Xt, Xv, y_train, y_valid, mappings, defaults, global_mean, agg_tables, final_cols

# # =====================================================
# # 8) Métricas y umbrales
# # =====================================================
# from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# def report_metrics(y_true, y_prob, thr, title=""):
#     y_hat = (y_prob >= thr).astype(int)
#     acc = accuracy_score(y_true, y_hat)
#     pre = precision_score(y_true, y_hat, zero_division=0)
#     rec = recall_score(y_true, y_hat, zero_division=0)
#     f1  = f1_score(y_true, y_hat, zero_division=0)
#     auc = roc_auc_score(y_true, y_prob)
#     cm  = confusion_matrix(y_true, y_hat)
#     print(f"\n== {title} (thr={thr:.3f}) ==")
#     print(f"Accuracy: {acc:.4f} | Precision: {pre:.4f} | Recall: {rec:.4f} | F1: {f1:.4f} | ROC-AUC: {auc:.4f}")
#     print("CM [TN, FP; FN, TP]:\n", cm)
#     return dict(acc=acc, pre=pre, rec=rec, f1=f1, auc=auc, thr=thr)

# def best_f1_threshold(y_true, y_prob):
#     best = {"thr": 0.5, "f1": -1}
#     for thr in np.linspace(0.05, 0.5, 46):
#         f1 = f1_score(y_true, (y_prob >= thr).astype(int), zero_division=0)
#         if f1 > best["f1"]:
#             best = {"thr": float(thr), "f1": float(f1)}
#     return best

# # =====================================================
# # 9) Entrenamiento modelos (LGBM / XGB / RF opcional)
# # =====================================================
# def train_lgbm(Xt, Xv, y_train, y_valid):
#     import lightgbm as lgb
#     neg = int((y_train==0).sum()); pos = int((y_train==1).sum())
#     spw = max(neg / max(pos,1), 1.0)
#     params = dict(
#         objective="binary",
#         learning_rate=0.05,
#         n_estimators=12000,
#         num_leaves=127,
#         min_child_samples=100,
#         subsample=0.8,
#         colsample_bytree=0.8,
#         reg_alpha=0.0,
#         reg_lambda=5.0,
#         n_jobs=-1,
#         random_state=42,
#         scale_pos_weight=spw
#     )
#     print("→ Entrenando LightGBM…")
#     model = lgb.LGBMClassifier(**params)
#     t0 = time.time()
#     model.fit(
#         Xt, y_train,
#         eval_set=[(Xv, y_valid)],
#         eval_metric="auc",
#         callbacks=[lgb.early_stopping(stopping_rounds=400), lgb.log_evaluation(200)]
#     )
#     secs = time.time() - t0
#     proba = model.predict_proba(Xv)[:,1]
#     auc   = roc_auc_score(y_valid, proba)
#     print(f"✓ Entrenado en {secs:.1f}s | best_iter={model.best_iteration_} | ROC-AUC valid={auc:.4f}")
#     return model, proba, auc

# def train_xgb(Xt, Xv, y_train, y_valid):
#     import xgboost as xgb
#     neg = int((y_train==0).sum()); pos = int((y_train==1).sum())
#     spw = max(neg / max(pos,1), 1.0)
#     dtrain = xgb.DMatrix(Xt, label=y_train)
#     dvalid = xgb.DMatrix(Xv, label=y_valid)
#     params = {
#         "objective": "binary:logistic",
#         "eval_metric": "auc",
#         "eta": 0.05,
#         "max_depth": 10,
#         "min_child_weight": 10,
#         "subsample": 0.8,
#         "colsample_bytree": 0.8,
#         "lambda": 5.0,
#         "alpha": 0.0,
#         "scale_pos_weight": spw,
#         "nthread": -1,
#         "seed": 42,
#         # velocidad/estabilidad
#         "tree_method": "hist"
#     }
#     print("→ Entrenando XGBoost…")
#     t0 = time.time()
#     watch = [ (dvalid, "valid") ]
#     bst = xgb.train(
#         params, dtrain,
#         num_boost_round=2000,
#         evals=watch,
#         early_stopping_rounds=400,
#         verbose_eval=200
#     )
#     secs = time.time() - t0
#     proba = bst.predict(dvalid, iteration_range=(0, bst.best_iteration+1))
#     auc   = roc_auc_score(y_valid, proba)
#     print(f"✓ XGB AUC valid={auc:.4f} | best_iter={bst.best_iteration} | tiempo={secs/60:.1f} min")
#     return bst, proba, auc

# def train_rf_optional(Xt, Xv, y_train, y_valid):
#     """RF opcional: desactivado por defecto desde train_full(…, also_rf=False)."""
#     from sklearn.ensemble import RandomForestClassifier
#     neg = int((y_train==0).sum()); pos = int((y_train==1).sum())
#     spw = max(neg / max(pos,1), 1.0)
#     class_w = {0: 1.0, 1: float(spw)}
#     params = dict(
#         n_estimators=400,
#         max_depth=24,
#         min_samples_leaf=2,
#         min_samples_split=4,
#         max_features="sqrt",
#         bootstrap=True,
#         n_jobs=-1,
#         random_state=42,
#         class_weight=class_w
#     )
#     print("→ Entrenando RandomForest (opcional)…")
#     t0 = time.time()
#     rf = RandomForestClassifier(**params).fit(Xt, y_train)
#     secs = time.time() - t0
#     proba = rf.predict_proba(Xv)[:,1]
#     auc   = roc_auc_score(y_valid, proba)
#     print(f"✓ RF AUC valid={auc:.4f} | tiempo={secs/60:.1f} min")
#     return rf, proba, auc

# # =====================================================
# # 10) Guardado de artefactos y metadata
# # =====================================================
# def save_artifacts(
#     model_lgbm, proba_lgbm, auc_lgbm,
#     model_xgb,  proba_xgb,  auc_xgb,
#     mappings, defaults, global_mean, agg_tables, feature_order,
#     out_dir_models="models", out_dir_art="artifacts"
# ):
#     os.makedirs(out_dir_models, exist_ok=True)
#     os.makedirs(out_dir_art, exist_ok=True)

#     # 10.1 Modelos
#     lgbm_path = os.path.join(out_dir_models, "lgbm_retrasos.pkl")
#     dump(model_lgbm, lgbm_path)
#     print(f"✅ LGBM guardado en: {lgbm_path}")

#     xgb_path  = os.path.join(out_dir_models, "xgb_retrasos.json")
#     if model_xgb is not None:
#         model_xgb.save_model(xgb_path)
#         print(f"✅ XGB guardado en: {xgb_path}")

#     # 10.2 TE
#     te_path = os.path.join(out_dir_art, "target_encoding.pkl")
#     dump({"mappings": mappings, "defaults": defaults}, te_path)
#     print(f"✅ Target Encoding guardado en: {te_path}")

#     # 10.3 Agregados -> CSV
#     agg_index = {}
#     for pref, agg_df in agg_tables.items():
#         csvp = os.path.join(out_dir_art, f"agg_{pref}.csv")
#         agg_df.to_csv(csvp, index=False)
#         agg_index[pref] = csvp
#     print(f"✅ Agregados guardados: {len(agg_index)} tablas")

#     # 10.4 Orden de features
#     feat_path = os.path.join(out_dir_art, "feature_order.json")
#     with open(feat_path, "w", encoding="utf-8") as f:
#         json.dump({"feature_order": feature_order}, f, ensure_ascii=False, indent=2)
#     print(f"✅ Orden de features guardado en: {feat_path}")

#     # 10.5 Umbrales (F1 óptimo por modelo)
#     best_lgbm = best_f1_threshold(y_valid, proba_lgbm) if proba_lgbm is not None else {"thr": None}
#     best_xgb  = best_f1_threshold(y_valid, proba_xgb)  if proba_xgb  is not None else {"thr": None}

#     # 10.6 Metadata global
#     meta = {
#         "created_at": time.strftime("%Y-%m-%d %H:%M:%S"),
#         "global_mean": float(global_mean),
#         "models": {
#             "lgbm": {
#                 "path": lgbm_path,
#                 "auc_valid": float(auc_lgbm),
#                 "best_f1_thr": float(best_lgbm["thr"]) if best_lgbm["thr"] is not None else None,
#                 "best_iteration": int(getattr(model_lgbm, "best_iteration_", -1))
#             },
#             "xgb": {
#                 "path": xgb_path if model_xgb is not None else None,
#                 "auc_valid": float(auc_xgb) if auc_xgb is not None else None,
#                 "best_f1_thr": float(best_xgb["thr"]) if (proba_xgb is not None and best_xgb["thr"] is not None) else None
#             }
#         },
#         "feature_order_path": feat_path,
#         "agg_tables": agg_index,
#         "te_cols": TE_COLS,
#         "final_feature_order": feature_order,
#         "split": {"train_months":"1-9","valid_months":"10-12"}
#     }
#     meta_path = os.path.join(out_dir_art, "metadata.json")
#     with open(meta_path, "w", encoding="utf-8") as f:
#         json.dump(meta, f, ensure_ascii=False, indent=2)
#     print(f"✅ Metadatos guardados en: {meta_path}")
#     return {"meta_path": meta_path, "lgbm_path": lgbm_path, "xgb_path": xgb_path}

# # =====================================================
# # 11) Orquestador de entrenamiento completo
# # =====================================================
# def train_full(path_csv: str, train_models=("lgbm","xgb"), also_rf=False):
#     # 11.1 Carga y features base
#     df = load_flights(path_csv)
#     df = add_route(df)
#     df = ensure_time_features(df)  # respeta lo que ya viene en CSV
#     df = ensure_distance(df)

#     # 11.2 Matrices
#     Xt, Xv, y_train, y_valid, mappings, defaults, global_mean, agg_tables, final_cols = make_matrices(df)
#     print(f"Shapes -> Xt: {Xt.shape} | Xv: {Xv.shape} | y_train rate={y_train.mean():.4f} | y_valid rate={y_valid.mean():.4f}")

#     # 11.3 Entrenamientos
#     model_lgbm = proba_lgbm = auc_lgbm = None
#     model_xgb  = proba_xgb  = auc_xgb  = None

#     if "lgbm" in train_models:
#         model_lgbm, proba_lgbm, auc_lgbm = train_lgbm(Xt, Xv, y_train, y_valid)
#         # Umbrales de reporte
#         _ = report_metrics(y_valid, proba_lgbm, 0.5, "LGBM Base 0.5")
#         best = best_f1_threshold(y_valid, proba_lgbm)
#         _ = report_metrics(y_valid, proba_lgbm, best["thr"], "LGBM Mejor F1")
#         # objetivos negocio (aprox)
#         # precisión ≳ 0.30
#         thr_prec = 0.229
#         _ = report_metrics(y_valid, proba_lgbm, thr_prec, "LGBM Precisión ≳ 0.30 (aprox)")
#         # recall ≥ 0.70 (barrido rápido)
#         thr_rec = 0.200
#         _ = report_metrics(y_valid, proba_lgbm, thr_rec, "LGBM Recall ≥ 0.70 (aprox)")

#     if "xgb" in train_models:
#         model_xgb, proba_xgb, auc_xgb = train_xgb(Xt, Xv, y_train, y_valid)
#         _ = report_metrics(y_valid, proba_xgb, 0.5, "XGB Base 0.5")
#         best = best_f1_threshold(y_valid, proba_xgb)
#         _ = report_metrics(y_valid, proba_xgb, best["thr"], "XGB Mejor F1")

#     if also_rf:
#         _rf, _rf_p, _rf_auc = train_rf_optional(Xt, Xv, y_train, y_valid)
#         best = best_f1_threshold(y_valid, _rf_p)
#         _ = report_metrics(y_valid, _rf_p, best["thr"], "RF Mejor F1 (opcional)")

#     # 11.4 Guardado de artefactos
#     arts = save_artifacts(
#         model_lgbm, proba_lgbm, auc_lgbm,
#         model_xgb,  proba_xgb,  auc_xgb,
#         mappings, defaults, global_mean, agg_tables, final_cols
#     )
#     return {
#         "lgbm_auc": auc_lgbm, "xgb_auc": auc_xgb,
#         "models_paths": arts, "features": final_cols
#     }

# # =====================================================
# # 12) Inferencia: score_retraso(df_raw)
# #     - Carga artefactos, rehace TE y agregados, alinea features, predice
# # =====================================================
# def score_retraso(df_raw: pd.DataFrame, model="lgbm", threshold=None,
#                   models_dir="models", art_dir="artifacts"):
#     # Cargar metadata
#     with open(os.path.join(art_dir, "metadata.json"), "r", encoding="utf-8") as f:
#         meta = json.load(f)

#     # Feature order
#     with open(meta["feature_order_path"], "r", encoding="utf-8") as f:
#         feature_order = json.load(f)["feature_order"]

#     # TE
#     te = load(os.path.join(art_dir, "target_encoding.pkl"))
#     mappings, defaults = te["mappings"], te["defaults"]

#     # Agregados guardados
#     agg_tables = {}
#     for pref, path in meta["agg_tables"].items():
#         agg_tables[pref] = pd.read_csv(path)

#     # Preparar entrada cruda → mismas features
#     df = df_raw.copy()
#     df = add_route(df)
#     df = ensure_time_features(df)   # respeta si ya trae MINUTO_DIA_SALIDA / SALIDA_SIN/COS
#     df = ensure_distance(df)

#     # Base + TE_COLS presentes
#     need_cols = set(BASE_FEATURES + TE_COLS)
#     miss = need_cols - set(df.columns)
#     if miss:
#         raise KeyError(f"Entrada de inferencia incompleta; faltan: {sorted(miss)}")

#     X = df[BASE_FEATURES + TE_COLS].copy()

#     # Agregados (join con CSVs persistidos)
#     # NOTA: para inferencia futura, estas tablas deberían alinearse con el histórico de entrenamiento
#     global_mean = float(meta["global_mean"])
#     # Orden y prefijos del entrenamiento
#     agg_plan = [
#         (["AIRLINE"], "AIR"),
#         (["DESTINATION_AIRPORT"], "DES"),
#         (["ORIGIN_AIRPORT"], "ORI"),
#         (["RUTA"], "RUTA"),
#         (["RUTA","HORA_SALIDA"], "RUTA_HORA"),
#     ]
#     for keys, pref in agg_plan:
#         agg = agg_tables[pref]
#         X = X.merge(agg, on=keys, how="left")
#         X[f"{pref}_rate"] = X[f"{pref}_rate"].fillna(global_mean).astype("float32")
#         X[f"{pref}_n"]    = X[f"{pref}_n"].fillna(0).astype("float32")

#     # TE
#     for c, mp in mappings.items():
#         newc = f"{c}_TE"
#         X[newc] = X[c].map(mp).astype("float32").fillna(defaults[c]).astype("float32")

#     # Orden final
#     X = X[feature_order].astype("float32")

#     # Cargar modelo
#     if model.lower() == "lgbm":
#         mdl = load(meta["models"]["lgbm"]["path"])
#         proba = mdl.predict_proba(X)[:,1]
#         thr   = threshold if threshold is not None else meta["models"]["lgbm"]["best_f1_thr"]
#     elif model.lower() == "xgb":
#         import xgboost as xgb
#         bst = xgb.Booster()
#         bst.load_model(meta["models"]["xgb"]["path"])
#         dX = xgb.DMatrix(X)
#         proba = bst.predict(dX, iteration_range=(0, getattr(bst, "best_iteration", 0)+1))
#         thr   = threshold if threshold is not None else meta["models"]["xgb"]["best_f1_thr"]
#     else:
#         raise ValueError("Modelo no soportado en inferencia: usa 'lgbm' o 'xgb'.")

#     thr = 0.5 if thr is None else float(thr)
#     yhat = (proba >= thr).astype(int)
#     return proba, yhat, X

# # =====================================================
# # === EJECUCIÓN (ejemplo) =============================
# # =====================================================
# if __name__ == "__main__":
#     # A) ENTRENAMIENTO COMPLETO (ajusta la ruta)
#     BASE_DIR = os.path.dirname(os.path.abspath(__file__)) if "__file__" in globals() else os.getcwd()
#     DATA_DIR = BASE_DIR
#     # csv_path = os.path.join(DATA_DIR, "flights_clean.csv")
#     csv_path = r"D:\OneDrive\DOCUMENTOS\Personales\2024\uniandes\8 S\seminario\g11-caso-estudio-flights\data\processed\flights_clean.csv"

#     print("\n===== ENTRENAMIENTO =====")
#     results = train_full(csv_path, train_models=("lgbm","xgb"), also_rf=False)
#     print("Resultados:", results)

#     # B) INFERENCIA (smoke test con un ejemplo)
#     print("\n===== INFERENCIA (smoke test) =====")
#     ejemplo = pd.DataFrame([dict(
#         MONTH=10, DAY_OF_WEEK=5,
#         AIRLINE="AA",
#         ORIGIN_AIRPORT="JFK", DESTINATION_AIRPORT="LAX",
#         SCHEDULED_DEPARTURE=830,  # HHMM (por si requieres derivar MINUTO_DIA_SALIDA)
#         ORIGEN_LAT=40.6413, ORIGEN_LON=-73.7781,
#         DEST_LAT=33.9416, DEST_LON=-118.4085,
#         # Si ya trajeras estas columnas, se respetan:
#         # MINUTO_DIA_SALIDA= ... , SALIDA_SIN=..., SALIDA_COS=..., HORA_SALIDA=...
#     )])

#     proba, clase, X_infer = score_retraso(ejemplo, model="lgbm")
#     print(pd.DataFrame({"proba_retraso": proba.round(4), "pred_clase": clase}))


📒 Celda 1 — Importaciones y configuració

In [6]:
# ================================
# 1) Importaciones y configuración
# ================================
import os, json, time, warnings
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

import lightgbm as lgb
import xgboost as xgb
import joblib

warnings.filterwarnings("ignore")

# Rutas base (ajusta si lo necesitas)
PROJECT_ROOT = Path(r"D:\OneDrive\DOCUMENTOS\Personales\2024\uniandes\8 S\seminario\g11-caso-estudio-flights")
DATA_DIR     = PROJECT_ROOT / "data" / "processed"
MODELS_DIR   = PROJECT_ROOT / "models"
ARTIF_DIR    = PROJECT_ROOT / "artifacts"

MODELS_DIR.mkdir(parents=True, exist_ok=True)
ARTIF_DIR.mkdir(parents=True, exist_ok=True)

# Ruta ABSOLUTA al CSV (tu ruta)
csv_path = r"D:\OneDrive\DOCUMENTOS\Personales\2024\uniandes\8 S\seminario\g11-caso-estudio-flights\data\processed\flights_clean.csv"

print("Usando CSV:", csv_path)


Usando CSV: D:\OneDrive\DOCUMENTOS\Personales\2024\uniandes\8 S\seminario\g11-caso-estudio-flights\data\processed\flights_clean.csv


📒 Celda 2 — Utilidades (downcast, haversine, helpers)

In [7]:
# ==========================================
# 2) Utilidades: downcast, haversine, helpers
# ==========================================

def downcast_numeric(df: pd.DataFrame) -> pd.DataFrame:
    """Reduce memoria en numéricos sin cambiar valores."""
    for c in df.select_dtypes(include=["int64","int32"]).columns:
        df[c] = pd.to_numeric(df[c], downcast="integer")
    for c in df.select_dtypes(include=["float64","float32"]).columns:
        df[c] = pd.to_numeric(df[c], downcast="float")
    return df

def haversine_km(lat1, lon1, lat2, lon2):
    """Distancia haversine en KM (vect.)"""
    R = 6371.0
    lat1 = np.radians(lat1); lon1 = np.radians(lon1)
    lat2 = np.radians(lat2); lon2 = np.radians(lon2)
    dlat = lat2 - lat1; dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    return 2*R*np.arcsin(np.sqrt(a))


📒 Celda 3 — Carga y features (respetando columnas ya presentes)

In [8]:
# ===========================================================
# 3) Carga flights_clean y features (NO recalcula si ya existen)
# ===========================================================

RAW_REQ_COLS = [
    "MONTH","DAY","DAY_OF_WEEK",
    "AIRLINE","ORIGIN_AIRPORT","DESTINATION_AIRPORT",
    "SCHEDULED_DEPARTURE","SCHEDULED_ARRIVAL",
    "ORIGEN_LAT","ORIGEN_LON","DEST_LAT","DEST_LON",
    "RETRASADO_LLEGADA"
]

# Features que podríamos derivar, pero respeta si ya vienen calculadas en el CSV
DERIVED_COLS = [
    "MINUTO_DIA_SALIDA","MINUTO_DIA_LLEGADA",
    "SALIDA_SIN","SALIDA_COS","LLEGADA_SIN","LLEGADA_COS",
    "HORA_SALIDA","HORA_LLEGADA",
    "DISTANCIA"
]

def load_flights(path_csv: str) -> pd.DataFrame:
    t0 = time.time()
    df = pd.read_csv(path_csv)
    print(f"✓ Cargado: {df.shape} | en {time.time()-t0:.1f}s")

    # Downcast seguro
    df = downcast_numeric(df)

    # Validación mínima
    for c in RAW_REQ_COLS:
        if c not in df.columns:
            print(f"⚠️ Falta columna esperada en CSV: {c}")

    return df

def add_route(df: pd.DataFrame) -> pd.DataFrame:
    """RUTA = ORIGIN -> DEST"""
    if "RUTA" not in df.columns:
        df["RUTA"] = (df["ORIGIN_AIRPORT"].astype(str) + "_" + df["DESTINATION_AIRPORT"].astype(str))
    return df

def ensure_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """Si faltan señales temporales, las crea. Si ya existen, no toca nada."""
    if "MINUTO_DIA_SALIDA" not in df.columns and "SCHEDULED_DEPARTURE" in df.columns:
        # SCHEDULED_DEPARTURE viene HHMM; construimos minuto del día
        hh = (df["SCHEDULED_DEPARTURE"] // 100).clip(0,23)
        mm = (df["SCHEDULED_DEPARTURE"] % 100).clip(0,59)
        df["MINUTO_DIA_SALIDA"] = hh*60 + mm

    # Señales sin/cos por salida
    if ("SALIDA_SIN" not in df.columns or "SALIDA_COS" not in df.columns) and "MINUTO_DIA_SALIDA" in df.columns:
        rad = 2*np.pi*(df["MINUTO_DIA_SALIDA"].astype(float)/(24*60))
        df["SALIDA_SIN"] = np.sin(rad).astype("float32")
        df["SALIDA_COS"] = np.cos(rad).astype("float32")

    # Hora de salida si falta
    if "HORA_SALIDA" not in df.columns and "SCHEDULED_DEPARTURE" in df.columns:
        df["HORA_SALIDA"] = (df["SCHEDULED_DEPARTURE"] // 100).clip(0,23).astype("int16")

    return df

def ensure_distance(df: pd.DataFrame) -> pd.DataFrame:
    """Asegura DISTANCIA: usa DISTANCE si está, si no calcula haversine."""
    if "DISTANCIA" not in df.columns:
        if "DISTANCE" in df.columns:
            df["DISTANCIA"] = pd.to_numeric(df["DISTANCE"], errors="coerce")
        else:
            df["DISTANCIA"] = haversine_km(df["ORIGEN_LAT"], df["ORIGEN_LON"], df["DEST_LAT"], df["DEST_LON"])
    return df


📒 Celda 4 — Target Encoding (con suavizado) y Agregados históricos

In [9]:
# ===================================================
# 4) Target Encoding (suavizado) y Agregados históricos
# ===================================================

# Columnas categóricas a codificar por TE
TE_COLS = ["AIRLINE","ORIGIN_AIRPORT","DESTINATION_AIRPORT","RUTA"]

def build_target_encoding(df, cols=TE_COLS, target="RETRASADO_LLEGADA", smooth=20):
    """
    Retorna 'mappings' (dict por col -> dict categoría:media_suavizada)
            y 'defaults' (dict por col -> media global) para inferencia.
    """
    mappings, defaults = {}, {}
    global_mean = df[target].mean()
    for c in cols:
        g = df.groupby(c)[target]
        m = g.mean()
        n = g.size()
        # suavizado tipo m-estimate
        enc = (m*n + global_mean*smooth) / (n + smooth)
        mappings[c] = enc.to_dict()
        defaults[c] = float(global_mean)
    return mappings, defaults

def apply_target_encoding(df, mappings, defaults, cols=TE_COLS, suffix="_TE"):
    """Aplica TE usando mappings/defaults (para train/valid y luego inferencia)."""
    out = df.copy()
    for c in cols:
        m = mappings.get(c, {})
        d = defaults.get(c, np.nan)
        out[c+suffix] = out[c].map(m).fillna(d).astype("float32")
    return out

# -------- Agregados históricos (sin fuga) --------

# Especificaciones: (keys, prefijo)
AGGS_SPECS = [
    (["AIRLINE"], "AIR"),
    (["DESTINATION_AIRPORT"], "DES"),
    (["ORIGIN_AIRPORT"], "ORI"),
    (["RUTA"], "RUTA"),
    (["RUTA","HORA_SALIDA"], "RUTA_HORA"),
]

def build_agg(df, keys, target="RETRASADO_LLEGADA", pref="X", smooth=20):
    """
    Construye tabla de agregados con media suavizada y conteo:
    -> columnas: keys + [f"{pref}_rate", f"{pref}_n"]
    """
    g = df.groupby(keys)[target]
    stats = g.mean()
    cnts  = g.size()
    global_mean = df[target].mean()
    rate = (stats*cnts + global_mean*smooth) / (cnts + smooth)
    out = rate.reset_index().rename(columns={target: f"{pref}_rate"})
    out = out.merge(cnts.rename(f"{pref}_n").reset_index(), on=keys, how="left")
    return out

def apply_aggregates(df, agg_tables):
    """
    Hace left-join de cada tabla de agregados en df.
    agg_tables: dict {pref -> (keys, agg_df)}
    """
    out = df.copy()
    for pref, (keys, agg_df) in agg_tables.items():
        out = out.merge(agg_df, on=keys, how="left")
        # Si quedan NaN en rate/n, rellena cero (categorías nuevas)
        out[f"{pref}_rate"] = out[f"{pref}_rate"].fillna(out[f"{pref}_rate"].mean()).astype("float32")
        out[f"{pref}_n"]    = out[f"{pref}_n"].fillna(0).astype("float32")
    return out


📒 Celda 5 — Construcción de matrices (train/valid), TE + agregados y selección de features

In [10]:
# ============================================================
# 5) Construcción de matrices, TE + agregados y selección final
# ============================================================

FEATURES_BASE = [
    # temporales
    "MONTH","DAY_OF_WEEK","HORA_SALIDA","MINUTO_DIA_SALIDA","SALIDA_SIN","SALIDA_COS",
    # distancia
    "DISTANCIA",
]

def build_matrices(df: pd.DataFrame):
    """
    Split temporal sin fuga:
      - TRAIN: MONTH in [1..9]
      - VALID: MONTH in [10..12]
    Aplica TE (mappings con TRAIN) y agregados (con TRAIN).
    Devuelve: X_train_model, X_valid_model, y_train, y_valid, metadata útil.
    """
    # 1) Asegurar columnas clave
    df = add_route(df)
    df = ensure_time_features(df)
    df = ensure_distance(df)

    # 2) Split temporal
    train_mask = df["MONTH"].between(1, 9)
    valid_mask = df["MONTH"].between(10, 12)

    v_train = df.loc[train_mask].copy()
    v_valid = df.loc[valid_mask].copy()

    y_train = v_train["RETRASADO_LLEGADA"].astype("int8")
    y_valid = v_valid["RETRASADO_LLEGADA"].astype("int8")

    # 3) Target Encoding (con TRAIN)
    mappings, defaults = build_target_encoding(v_train, cols=TE_COLS, target="RETRASADO_LLEGADA", smooth=20)
    v_train = apply_target_encoding(v_train, mappings, defaults, cols=TE_COLS, suffix="_TE")
    v_valid = apply_target_encoding(v_valid, mappings, defaults, cols=TE_COLS, suffix="_TE")

    # 4) Agregados históricos (con TRAIN)
    agg_tables = {}
    for keys, pref in AGGS_SPECS:
        agg_df = build_agg(v_train, keys, target="RETRASADO_LLEGADA", pref=pref, smooth=20)
        agg_tables[pref] = (keys, agg_df)

    v_train = apply_aggregates(v_train, agg_tables)
    v_valid = apply_aggregates(v_valid, agg_tables)

    # 5) Selección de features finales
    te_feats = [c+"_TE" for c in TE_COLS]
    agg_feats = []
    for _, pref in AGGS_SPECS:
        agg_feats += [f"{pref}_rate", f"{pref}_n"]

    all_feats = FEATURES_BASE + te_feats + agg_feats

    # Alinear y asegurar tipos
    X_train_model = v_train[all_feats].copy()
    X_valid_model = v_valid[all_feats].copy()

    for c in X_train_model.select_dtypes(include=["float64"]).columns:
        X_train_model[c] = X_train_model[c].astype("float32")
    for c in X_valid_model.select_dtypes(include=["float64"]).columns:
        X_valid_model[c] = X_valid_model[c].astype("float32")

    meta = dict(
        features=all_feats,
        mappings=mappings,
        defaults=defaults,
        agg_tables=agg_tables,
        train_rate=float(y_train.mean()),
        valid_rate=float(y_valid.mean())
    )
    print("Shapes ->", X_train_model.shape, X_valid_model.shape, "| rate train", meta["train_rate"], "| rate valid", meta["valid_rate"])
    return X_train_model, X_valid_model, y_train, y_valid, meta


📒 Celda 6 — Entrenamiento LightGBM

In [11]:
# ======================================
# 6) Entrenamiento LightGBM (validación)
# ======================================

def train_lgbm(Xtr, ytr, Xv, yv):
    neg = int((ytr==0).sum()); pos = int((ytr==1).sum())
    scale_pos_weight = max(neg / max(pos,1), 1.0)

    params = dict(
        objective="binary",
        learning_rate=0.05,
        n_estimators=12000,
        num_leaves=127,
        min_child_samples=100,
        subsample=0.8, colsample_bytree=0.8,
        reg_alpha=0.0, reg_lambda=5.0,
        n_jobs=-1, random_state=42,
        scale_pos_weight=scale_pos_weight
    )
    model = lgb.LGBMClassifier(**params)
    t0 = time.time()
    model.fit(
        Xtr, ytr,
        eval_set=[(Xv, yv)],
        eval_metric="auc",
        callbacks=[lgb.early_stopping(stopping_rounds=400), lgb.log_evaluation(200)]
    )
    secs = time.time()-t0
    proba = model.predict_proba(Xv)[:,1]
    auc   = roc_auc_score(yv, proba)
    print(f"✓ LGBM entrenado en {secs:.1f}s | best_iter={model.best_iteration_} | AUC valid={auc:.4f}")
    return model, proba, auc


📒 Celda 7 — Entrenamiento XGBoost (pequeña búsqueda)

In [12]:
# ======================================
# 7) Entrenamiento XGBoost (grid pequeño)
# ======================================

def train_xgb(Xtr, ytr, Xv, yv):
    dtr = xgb.DMatrix(Xtr, label=ytr)
    dv  = xgb.DMatrix(Xv,  label=yv)

    neg = int((ytr==0).sum()); pos = int((ytr==1).sum())
    spw = max(neg/max(pos,1), 1.0)

    grids = [
        dict(eta=0.05, max_depth=10, min_child_weight=10, subsample=0.8, colsample_bytree=0.8),
        dict(eta=0.05, max_depth=8,  min_child_weight=5,  subsample=0.8, colsample_bytree=0.8),
        dict(eta=0.03, max_depth=10, min_child_weight=10, subsample=0.9, colsample_bytree=0.9),
    ]
    best = {"auc":-1, "model":None, "proba":None, "params":None, "best_iter":None}

    for g in grids:
        params = dict(
            objective="binary:logistic", eval_metric="auc",
            eta=g["eta"], max_depth=g["max_depth"], min_child_weight=g["min_child_weight"],
            subsample=g["subsample"], colsample_bytree=g["colsample_bytree"],
            reg_lambda=5.0, reg_alpha=0.0,
            scale_pos_weight=spw, nthread=-1, seed=42
        )
        print("Probando XGB:", params)
        t0 = time.time()
        mdl = xgb.train(
            params, dtr,
            num_boost_round=4000,
            evals=[(dv,"valid")],
            early_stopping_rounds=400,
            verbose_eval=400
        )
        secs = time.time()-t0
        proba = mdl.predict(dv, iteration_range=(0, mdl.best_iteration+1))
        auc   = roc_auc_score(yv, proba)
        print(f"AUC valid={auc:.4f} | best_iter={mdl.best_iteration} | tiempo={secs/60:.1f} min\n")
        if auc > best["auc"]:
            best = {"auc": auc, "model": mdl, "proba": proba, "params": params, "best_iter": mdl.best_iteration}
    print("=== XGB MEJOR ===")
    print(best["params"], "| AUC:", best["auc"], "| best_iter:", best["best_iter"])
    return best["model"], best["proba"], best["auc"], best["params"], best["best_iter"]


📒 Celda “7.5” — Random Forest (entrenar y evaluar)

In [14]:
# ======================================
# 7.5) Random Forest (sklearn): entrenamiento
# ======================================
import time
from sklearn.ensemble import RandomForestClassifier

def train_rf(Xtr, ytr, Xv, yv):
    """
    Entrena un RandomForest con parámetros conservadores para evitar OOM (memoria).
    Devuelve: (modelo, proba_valid, auc_valid, params_usados)
    """
    # Desbalance: usa ponderación automática por clase en cada bootstrap
    base_params = dict(
        n_estimators=300,               # si tienes más tiempo/RAM: 400–600
        max_depth=22,                   # limitar profundidad evita explosión de memoria
        min_samples_leaf=3,
        min_samples_split=6,
        max_features="sqrt",
        bootstrap=True,
        max_samples=0.6,                # submuestreo por árbol (reduce RAM y overfitting)
        class_weight="balanced_subsample",
        n_jobs=-1,
        random_state=42,
    )

    rf = RandomForestClassifier(**base_params)
    t0 = time.time()
    rf.fit(Xtr, ytr)
    secs = time.time() - t0

    proba = rf.predict_proba(Xv)[:, 1]
    auc   = roc_auc_score(yv, proba)
    print(f"✓ RF entrenado en {secs/60:.1f} min | AUC valid={auc:.4f}")
    return rf, proba, auc, base_params


📒 Celda 8 — Métricas rápidas y elección del mejor modelo

In [15]:
# ======================================================
# 8) Métricas, búsqueda de umbral y elección del mejor
# ======================================================

def report_metrics(y_true, y_prob, thr, title=""):
    y_hat = (y_prob >= thr).astype(int)
    acc = accuracy_score(y_true, y_hat)
    pre = precision_score(y_true, y_hat, zero_division=0)
    rec = recall_score(y_true, y_hat, zero_division=0)
    f1  = f1_score(y_true, y_hat, zero_division=0)
    auc = roc_auc_score(y_true, y_prob)
    cm  = confusion_matrix(y_true, y_hat)
    print(f"\n== {title} (thr={thr:.3f}) ==")
    print(f"Accuracy: {acc:.4f} | Precision: {pre:.4f} | Recall: {rec:.4f} | F1: {f1:.4f} | ROC-AUC: {auc:.4f}")
    print("CM [TN, FP; FN, TP]:\n", cm)
    return dict(acc=acc, pre=pre, rec=rec, f1=f1, auc=auc, thr=thr)

def best_f1_threshold(y_true, y_prob, lo=0.05, hi=0.5, steps=46):
    best = {"thr":0.5, "f1":-1}
    for thr in np.linspace(lo, hi, steps):
        y_hat = (y_prob >= thr).astype(int)
        f1 = f1_score(y_true, y_hat, zero_division=0)
        if f1 > best["f1"]:
            best = {"thr": float(thr), "f1": float(f1)}
    return best


📒 Celda 9 — Entrenamiento completo (carga → matrices → LGBM & XGB → guardado)

In [16]:
# ===========================================================
# 9) Pipeline de entrenamiento completo y guardado de artefactos
#     (ahora soporta: LGBM, XGB y RF)
# ===========================================================

def train_full(path_csv: str, train_models=("lgbm","xgb","rf")):
    # 9.1 Carga y features base (respeta columnas ya calculadas en el CSV)
    df = load_flights(path_csv)
    df = add_route(df)
    df = ensure_time_features(df)
    df = ensure_distance(df)

    # 9.2 Construcción de matrices
    Xtr, Xv, ytr, yv, meta = build_matrices(df)
    feature_order = meta["features"]

    # 9.3 Entrenamientos según selección
    results = {}

    if "lgbm" in train_models:
        lgbm_model, lgbm_proba, lgbm_auc = train_lgbm(Xtr, ytr, Xv, yv)
        lgbm_best = best_f1_threshold(yv, lgbm_proba)
        report_metrics(yv, lgbm_proba, 0.5, "LGBM Base 0.5")
        report_metrics(yv, lgbm_proba, lgbm_best["thr"], "LGBM Mejor F1")
        results["lgbm"] = dict(auc=lgbm_auc, proba=lgbm_proba, model=lgbm_model, best_thr=lgbm_best)

    if "xgb" in train_models:
        xgb_model, xgb_proba, xgb_auc, xgb_params, xgb_best_iter = train_xgb(Xtr, ytr, Xv, yv)
        xgb_best = best_f1_threshold(yv, xgb_proba)
        report_metrics(yv, xgb_proba, 0.5, "XGB Base 0.5")
        report_metrics(yv, xgb_proba, xgb_best["thr"], "XGB Mejor F1")
        results["xgb"] = dict(auc=xgb_auc, proba=xgb_proba, model=xgb_model, best_thr=xgb_best,
                              params=xgb_params, best_iter=xgb_best_iter)

    if "rf" in train_models:
        rf_model, rf_proba, rf_auc, rf_params = train_rf(Xtr, ytr, Xv, yv)
        rf_best = best_f1_threshold(yv, rf_proba)
        report_metrics(yv, rf_proba, 0.5, "RF Base 0.5")
        report_metrics(yv, rf_proba, rf_best["thr"], "RF Mejor F1")
        results["rf"] = dict(auc=rf_auc, proba=rf_proba, model=rf_model, best_thr=rf_best, params=rf_params)

    # 9.4 Elegir mejor por AUC
    best_name = max(results.keys(), key=lambda k: results[k]["auc"])
    best_info = results[best_name]
    print(f"\n=== MEJOR MODELO: {best_name.upper()} | AUC={best_info['auc']:.4f} ===")

    # 9.5 Guardado de artefactos comunes
    te_path = ARTIF_DIR / "target_encoding.pkl"
    joblib.dump({"mappings": meta["mappings"], "defaults": meta["defaults"]}, te_path)

    agg_registry = {}
    for pref, (keys, agg_df) in meta["agg_tables"].items():
        out_csv = ARTIF_DIR / f"agg_{pref}.csv"
        agg_df.to_csv(out_csv, index=False)
        agg_registry[pref] = dict(keys=keys, path=str(out_csv))

    feat_path = ARTIF_DIR / "feature_order.json"
    with open(feat_path, "w", encoding="utf-8") as f:
        json.dump({"feature_order": feature_order}, f, ensure_ascii=False, indent=2)

    # 9.6 Guardar modelo según tipo
    if best_name == "lgbm":
        model_path = MODELS_DIR / "lgbm_retrasos.pkl"
        joblib.dump(best_info["model"], model_path)
        best_iter = int(getattr(best_info["model"], "best_iteration_", 0) or 0)

    elif best_name == "xgb":
        model_path = MODELS_DIR / "xgb_retrasos.model"
        best_info["model"].save_model(str(model_path))
        best_iter = int(best_info.get("best_iter", 0) or 0)

    else:  # rf
        model_path = MODELS_DIR / "rf_retrasos.pkl"
        joblib.dump(best_info["model"], model_path)
        best_iter = 0  # RF no tiene "best_iteration"

    meta_json = {
        "created_at": time.strftime("%Y-%m-%d %H:%M:%S"),
        "model_type": best_name,                      # "lgbm" | "xgb" | "rf"
        "model_path": str(model_path),
        "best_iter": best_iter,
        "auc_valid": float(best_info["auc"]),
        "best_thr_f1": float(best_info["best_thr"]["thr"]),
        "feature_order_path": str(feat_path),
        "te_path": str(te_path),
        "aggs": agg_registry,
        "te_cols": TE_COLS
    }
    meta_path = ARTIF_DIR / "metadata.json"
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta_json, f, ensure_ascii=False, indent=2)

    print("\n✅ Artefactos guardados:")
    print(" - Modelo:", model_path)
    print(" - TE:", te_path)
    print(" - Feature order:", feat_path)
    print(" - Metadatos:", meta_path)
    print(" - Agregados:", list(agg_registry.keys()))
    return dict(best=best_name, auc=best_info["auc"], thr=best_info["best_thr"]["thr"], model_path=str(model_path))


📒 Celda 10 (reemplazo) — Inferencia compatible con RF

In [17]:
# ===========================================================
# 10) Inferencia: carga artefactos y función de scoring (LGBM/XGB/RF)
# ===========================================================

def load_artifacts():
    with open(ARTIF_DIR / "metadata.json", "r", encoding="utf-8") as f:
        META = json.load(f)

    with open(ARTIF_DIR / "feature_order.json", "r", encoding="utf-8") as f:
        feature_order = json.load(f)["feature_order"]

    te_bundle = joblib.load(ARTIF_DIR / "target_encoding.pkl")
    mappings = te_bundle["mappings"]; defaults = te_bundle["defaults"]

    # Agregados
    agg_tables = {}
    for pref, info in META["aggs"].items():
        keys = info["keys"]
        agg_df = pd.read_csv(info["path"])
        agg_tables[pref] = (keys, agg_df)

    # Modelo
    mtype = META["model_type"]
    if mtype == "lgbm":
        model = joblib.load(META["model_path"])
    elif mtype == "xgb":
        booster = xgb.Booster()
        booster.load_model(META["model_path"])
        model = booster
    else:  # rf
        model = joblib.load(META["model_path"])

    return META, feature_order, mappings, defaults, agg_tables, model

def score_retraso(df_raw: pd.DataFrame, threshold=None, model_type=None):
    """
    df_raw: DataFrame con columnas mínimas (ver RAW_REQ_COLS).
    threshold: si None, usa best_thr_f1.
    model_type: si None, usa el del metadata (lgbm/xgb/rf).
    """
    META, feature_order, mappings, defaults, agg_tables, model = load_artifacts()
    if threshold is None:
        threshold = META["best_thr_f1"]
    if model_type is None:
        model_type = META["model_type"]

    # 1) Asegurar features base
    df = df_raw.copy()
    df = add_route(df)
    df = ensure_time_features(df)
    df = ensure_distance(df)

    # 2) Target Encoding
    df = apply_target_encoding(df, mappings, defaults, cols=META["te_cols"], suffix="_TE")

    # 3) Agregados históricos
    df = apply_aggregates(df, agg_tables)

    # 4) Matriz final
    X = df[feature_order].copy().astype("float32")

    # 5) Predicción por tipo
    if model_type == "lgbm":
        proba = model.predict_proba(X, num_iteration=getattr(model, "best_iteration_", None))[:,1]
    elif model_type == "xgb":
        dmat = xgb.DMatrix(X)
        best_iter = META.get("best_iter", 0) or 0
        proba = model.predict(dmat, iteration_range=(0, best_iter)) if best_iter > 0 else model.predict(dmat)
    else:  # rf
        proba = model.predict_proba(X)[:, 1]

    yhat = (proba >= threshold).astype(int)
    return proba, yhat, X


Para entrenar con los tres modelos y que el sistema elija automáticamente el mejor por AUC:

In [18]:
results = train_full(csv_path, train_models=("lgbm","xgb","rf"))
results


✓ Cargado: (5231130, 41) | en 46.9s
Shapes -> (4299046, 21) (932084, 21) | rate train 0.18733737671101913 | rate valid 0.17261212508743848
[LightGBM] [Info] Number of positive: 805372, number of negative: 3493674
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.470835 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3533
[LightGBM] [Info] Number of data points in the train set: 4299046, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.187337 -> initscore=-1.467405
[LightGBM] [Info] Start training from score -1.467405
Training until validation scores don't improve for 400 rounds
[200]	valid_0's auc: 0.598761	valid_0's binary_logloss: 0.557709
[400]	valid_0's auc: 0.598581	valid_0's binary_logloss: 0.558715
Early stopping, best iteration is:
[1]	valid_0's auc: 0.600624	valid_0's binary_logloss: 0.459715
✓ LGBM entrenado en 187.8s | best_iter=1 | AUC valid=0.6006

== LGBM Bas

{'best': 'xgb',
 'auc': 0.6062161779601237,
 'thr': 0.48,
 'model_path': 'D:\\OneDrive\\DOCUMENTOS\\Personales\\2024\\uniandes\\8 S\\seminario\\g11-caso-estudio-flights\\models\\xgb_retrasos.model'}

***Resultados*

✓ Cargado: (5231130, 41) | en 46.9s
Shapes -> (4299046, 21) (932084, 21) | rate train 0.18733737671101913 | rate valid 0.17261212508743848
[LightGBM] [Info] Number of positive: 805372, number of negative: 3493674
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.470835 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3533
[LightGBM] [Info] Number of data points in the train set: 4299046, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.187337 -> initscore=-1.467405
[LightGBM] [Info] Start training from score -1.467405
Training until validation scores don't improve for 400 rounds
[200]	valid_0's auc: 0.598761	valid_0's binary_logloss: 0.557709
[400]	valid_0's auc: 0.598581	valid_0's binary_logloss: 0.558715
Early stopping, best iteration is:
[1]	valid_0's auc: 0.600624	valid_0's binary_logloss: 0.459715
✓ LGBM entrenado en 187.8s | best_iter=1 | AUC valid=0.6006

== LGBM Base 0.5 (thr=0.500) ==
Accuracy: 0.8274 | Precision: 0.0000 | Recall: 0.0000 | F1: 0.0000 | ROC-AUC: 0.6006
CM [TN, FP; FN, TP]:
 [[771195      0]
 [160889      0]]

== LGBM Mejor F1 (thr=0.190) ==
Accuracy: 0.3441 | Precision: 0.1915 | Recall: 0.8691 | F1: 0.3139 | ROC-AUC: 0.6006
CM [TN, FP; FN, TP]:
 [[180860 590335]
 [ 21059 139830]]
Probando XGB: {'objective': 'binary:logistic', 'eval_metric': 'auc', 'eta': 0.05, 'max_depth': 10, 'min_child_weight': 10, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_lambda': 5.0, 'reg_alpha': 0.0, 'scale_pos_weight': 4.337963077931689, 'nthread': -1, 'seed': 42}
[0]	valid-auc:0.58792
[400]	valid-auc:0.59664
[402]	valid-auc:0.59666
AUC valid=0.6062 | best_iter=2 | tiempo=4.7 min

Probando XGB: {'objective': 'binary:logistic', 'eval_metric': 'auc', 'eta': 0.05, 'max_depth': 8, 'min_child_weight': 5, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_lambda': 5.0, 'reg_alpha': 0.0, 'scale_pos_weight': 4.337963077931689, 'nthread': -1, 'seed': 42}
[0]	valid-auc:0.58954
[400]	valid-auc:0.59806
[401]	valid-auc:0.59805
AUC valid=0.6059 | best_iter=2 | tiempo=4.0 min

Probando XGB: {'objective': 'binary:logistic', 'eval_metric': 'auc', 'eta': 0.03, 'max_depth': 10, 'min_child_weight': 10, 'subsample': 0.9, 'colsample_bytree': 0.9, 'reg_lambda': 5.0, 'reg_alpha': 0.0, 'scale_pos_weight': 4.337963077931689, 'nthread': -1, 'seed': 42}
[0]	valid-auc:0.58822
[400]	valid-auc:0.59670
[402]	valid-auc:0.59670
AUC valid=0.6041 | best_iter=2 | tiempo=4.5 min

=== XGB MEJOR ===
{'objective': 'binary:logistic', 'eval_metric': 'auc', 'eta': 0.05, 'max_depth': 10, 'min_child_weight': 10, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_lambda': 5.0, 'reg_alpha': 0.0, 'scale_pos_weight': 4.337963077931689, 'nthread': -1, 'seed': 42} | AUC: 0.6062161779601237 | best_iter: 2

== XGB Base 0.5 (thr=0.500) ==
Accuracy: 0.7111 | Precision: 0.2445 | Recall: 0.3224 | F1: 0.2781 | ROC-AUC: 0.6062
CM [TN, FP; FN, TP]:
 [[610958 160237]
 [109020  51869]]

== XGB Mejor F1 (thr=0.480) ==
Accuracy: 0.4841 | Precision: 0.2084 | Recall: 0.7106 | F1: 0.3222 | ROC-AUC: 0.6062
CM [TN, FP; FN, TP]:
 [[336857 434338]
 [ 46566 114323]]
✓ RF entrenado en 15.1 min | AUC valid=0.5922

== RF Base 0.5 (thr=0.500) ==
Accuracy: 0.7898 | Precision: 0.2574 | Recall: 0.1155 | F1: 0.1595 | ROC-AUC: 0.5922
CM [TN, FP; FN, TP]:
 [[717591  53604]
 [142305  18584]]

== RF Mejor F1 (thr=0.230) ==
Accuracy: 0.4813 | Precision: 0.2041 | Recall: 0.6917 | F1: 0.3152 | ROC-AUC: 0.5922
CM [TN, FP; FN, TP]:
 [[337293 433902]
 [ 49610 111279]]

=== MEJOR MODELO: XGB | AUC=0.6062 ===

✅ Artefactos guardados:
 - Modelo: D:\OneDrive\DOCUMENTOS\Personales\2024\uniandes\8 S\seminario\g11-caso-estudio-flights\models\xgb_retrasos.model
 - TE: D:\OneDrive\DOCUMENTOS\Personales\2024\uniandes\8 S\seminario\g11-caso-estudio-flights\artifacts\target_encoding.pkl
 - Feature order: D:\OneDrive\DOCUMENTOS\Personales\2024\uniandes\8 S\seminario\g11-caso-estudio-flights\artifacts\feature_order.json
 - Metadatos: D:\OneDrive\DOCUMENTOS\Personales\2024\uniandes\8 S\seminario\g11-caso-estudio-flights\artifacts\metadata.json
 - Agregados: ['AIR', 'DES', 'ORI', 'RUTA', 'RUTA_HORA']

{'best': 'xgb',
 'auc': 0.6062161779601237,
 'thr': 0.48,
 'model_path': 'D:\\OneDrive\\DOCUMENTOS\\Personales\\2024\\uniandes\\8 S\\seminario\\g11-caso-estudio-flights\\models\\xgb_retrasos.model'}